# YOLO V3

卷积提取特征本质, 矩阵解释:https://blog.csdn.net/IT__learning/article/details/119086012

#### batch normalization

* 用于将分布在较大空间内的数据整合到一个比较小的数据空间内, 减少经过非线性以后的数据饱和. 

* 建议在每一个卷积层和ReLU层之间都加入这个层

![bn](bn.png)

输入层是一个大范围内分布的离散数值, 先计算他们的算术平均值, 再计算方差, 经过正则化处理, $\epsilon$是一个极小量, 用于防止方差为零. 最后再进行一个线性预测, 并加入bias $\beta$. 其中, 需要机器学习的量仅为$\gamma$和$\beta$, 当$\gamma = \sigma, \beta = \mu$时, BN层将失去效果

#### 上采样和下采样

* 上采样是用于放大原图像, 使之像素点更多, 多采用内插值方法, 在原图的基础上插入合适的像素点完成.

* 下采样是用于缩小图像, 一般用于求一个区域内均值的方法把原图映射到下采样图样中

## 评价指标

![tp](TP.png)

* 准确率
  
  $$A = \frac{TP+TN}{P+N}$$
  反映预测正确的结果占全体样本的数量

* 精确度
  
  $$ P = \frac{TP}{TP+FP} $$
  反映预测正确的正类占据整个预测结果正类的比值

* 召回率
  
  $$R = \frac{TP}{TP+FN}$$
  反映正确判定的正例占据全体实际正例的比例

* AP与mAP
  
  AP指对于某一个分类, P-R曲线下的面积, mAP指对于所有分类任务的AP面积的平均值. AP和mAP面积越大效果越好, 其取值范围[0,1]

* IoU
  
  算法产生的预测框与实际标注的预测框相交面积除以相并面积的比值

## 锚框(先验框)

锚框在训练时作为基本单位, 同样, 为每一张真实图片打上的每一个标签框也作为一个样本参与训练. 一张图可能有多个样本.

传统锚框是以某一个像素点为中心, 指定长宽形成的一个包括多个像素点的矩形框, 对于同一个像素点可以生成很多锚框, 这些锚框可以具有不同的长宽比, 大小(一般由人手动设置长宽比, 锚框个数, 锚框大小), 在训练时, 通过卷积形成的特征图一般有多个维数, 那么对应其中某一个点的锚框就具有同样的特征维数.

训练在最终卷积层结束提取出特征图上使用, 需要将特征图映射到原图中去寻找相应的锚框. 并找出与原始标签框IoU最大的作为正类.  (此处标签可以有不同的类, 相应的锚框也具有对应的标签).

由此处得到了一个由定锚框样式对应到标签框的过程. 需要学习的是某一个物体标签框对应于哪一个锚点的哪一种锚框.

* 预测时为避免多个锚框同时被预测出来而相互重叠, 采用非极大抑制(non-maximum suppression, NMS)进行选择框.
  
  对一张图片的全部预测框, 把它预测对应物体类别的概率进行排序, 取最大的概率为相应的标签(前提是最大的概率需要大于某一个阈值). 对于同一标签的框们, 计算他们之间的IoU, 如果大于某一阈值, 舍弃对应概率比较小的框, 如果小于阈值则保留两个框

### YOLO采用锚框方法

#### 训练时

YOLO锚框参数: $t_x, t_y, t_w, t_h, conf$, 其中$t_x, t_y$意义是锚框中心坐标, 其值是相对于对应**cell感受野的左上角**对应的像素坐标差值. $t_x, t_y$是锚框的长和宽, $conf$是置信度

* 输入图像先进入卷积层提取特征并下采样减少窗口大小, 
* 对于卷积后特征图的每一个数据cell(这个cell在原图上感受野是一个$S\times S$的网格, 相当于将原图分割为多个尺寸为$S\times S$的网格拼接), 使用K-means聚类选取一定量的锚框个数.
* 针对某一个cell只会有一种标签种类输出--利用这些选取的锚框与原图标签框对比, 选取IoU最大的锚框作为预测框, 并将该框置为1, 其余锚框均为零. 并标记该锚框为对应种类
* 对所求得的锚框与真实锚框的损失函数进行回归迭代. 得到参数

#### 预测时

* 对每一个网格预测得到的锚框偏移参数$t_x, t_y, t_w, t_y$, 将其每个量送入Sigmoid函数处理得到一个(0,1)之间的数值. 那么预测锚框的中心坐标就是该锚框对应的网格左上角的坐标值加上Sigmoid处理后的值针对网格大小等比例放大后的数值. 长宽则是针对整张图片大小等比例放大的结果
  
  设网格左上角顶点坐标为$(x,y)$, Sigmoid处理后的值为$\sigma_x, \sigma_y, \sigma_w, \sigma_h$, 网格大小为$S\times S$, 整张图片的高宽为$H, W$那么最终锚框参数有
  $$B_x = x + S\times \sigma_x \\
  B_y = y + S\times \sigma_y \\
  B_w = W\times \sigma_w\\
  B_h = H\times \sigma_h
  $$
  对于最终的锚框, 需要进行NMS操作选取置信度最大的锚框

#### 锚框损失函数

![loss](lossfun.png)

以上是YOLOv1损失函数, YOLOv1只有两个锚框. 式中$\mathbb{1}_{ij}^{obj}$在锚框含有物体时取1, 不含有取0, 而$\mathbb{1}_{ij}^{noobj}$与前者相反, 在取到物体时为0, 没有取到为1
**注意: 此处与置信度不同, 某一个锚框可能置信度为0但含有物体.**

式中双重求和$\sum_{i=0}^{S^2} \sum_{j=0}^{B}$指对cell的每一个锚框都进行遍历(无论是否取到物体), S是cell个数, B是每个cell生成锚框个数

式中$\lambda_{coord}$是包含目标的网格权重一般设置为比较大, $\lambda_{noobj}$是不包含物体的网格权重, 一般设置为比较小但是不为零. (为零将使反向传播梯度过大)

式中$C_i$是预测锚框置信度. $\hat {C_i}$是真实置信度, 对于真实置信度, 如果当前锚框在之前cell的多锚框选择中被选中负责该物体的检测, 那么此处$\hat C_i$取值为1, 如果不是, 取值为0(与cell选择锚框中$conf$参数一致). 而预测锚框置信度计算方式如下
$$C_i = Pr(Class_i|Object)\times Pr(Object)\times {IoU}_{pred}^{truth} = Pr(Class_i)\times {IoU}_{pred}^{truth}
$$
式中$Pr(Object)$是含有物体的概率, $Pr(Class_i|Object)$是在含有物体的情况下是$Class_i$类的条件概率

实际应用中的概率计算: 负责检测该物体的锚框$Pr(Object) = 1$, 而$Pr(Class_i|Object)$为卷积网络得到判定为该类的置信度. 而不负责检测检测该物体的$Pr(Object) = 0$